In [1]:
from langchain.tools import DuckDuckGoSearchResults

search = DuckDuckGoSearchResults()
result = search.run(
    "Research how to use the requests library in Python. Use your tools to search and summarize content into a guide on how to use the requests library."
)
print(result)

ModuleNotFoundError: No module named 'langchain'

In [8]:
# source https://dev.to/timesurgelabs/how-to-make-an-ai-agent-in-10-minutes-with-langchain-3i2n
import requests
from bs4 import BeautifulSoup
from langchain.tools import DuckDuckGoSearchResults
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.agents import AgentExecutor, create_react_agent
from langchain.tools import StructuredTool, tool
import os

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"] = "{YOUR_LANGCHAIN_APIKEY}"
os.environ["LANGCHAIN_PROJECT"] = "{YOUR_LANGSMITH_PROJECT}"
os.environ["OPENAI_API_KEY"] = "{YOUR_OPENAI_KEY}"
os.environ["SERPER_API_KEY"] = "{YOUR_SERPER_APIKEY}"

model = ChatOpenAI(model="gpt-3.5-turbo-16k")

ddg_search = DuckDuckGoSearchResults()
HEADERS = {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:90.0) Gecko/20100101 Firefox/90.0"
}


def parse_html(content) -> str:
    soup = BeautifulSoup(content, "html.parser")
    text_content_with_links = soup.get_text()[:3000]
    return text_content_with_links


@tool
def web_fetch_tool(url: str) -> str:
    """Useful to fetches the contents of a web page"""
    if isinstance(url, list):
        url = url[0]
    print("Fetch_web_page URL :", url)
    response = requests.get(url, headers=HEADERS)
    return parse_html(response.content)


summarization_chain = LLMChain(
    llm=model,
    prompt=PromptTemplate.from_template("Summarize the following content: {content}"),
)
summarize_tool = StructuredTool.from_function(
    func=summarization_chain.run,
    name="Summarizer",
    description="Useful to summarizes a web page",
)

tools = [ddg_search, web_fetch_tool, summarize_tool]

template = """Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}"""

prompt = PromptTemplate.from_template(template)

agent = create_react_agent(model, tools, prompt)
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
    return_intermediate_steps=True,
    handle_parsing_errors=True,
)

question = "Tell me about best Korean reastaurant in Seoul.\
Use search tool to find the information.\
To get the details, please fetch the contents from the web sites.\
Summarize the details in 1000 words."

print(agent_executor.invoke({"input": question}))



> Entering new AgentExecutor chain...
I need to use the DuckDuckGo Results JSON tool to search for the best Korean restaurant in Seoul. Then, I can use the web_fetch_tool to fetch the contents of the websites that provide information about the restaurant. Finally, I can use the Summarizer tool to summarize the details in 1000 words.
Action: DuckDuckGo Results JSON
Action Input: "best Korean restaurant in Seoul"

/Users/terrycho/anaconda3/lib/python3.11/site-packages/langchain_community/utilities/duckduckgo_search.py:47: UserWarning: DDGS running in an async loop. This may cause errors. Use AsyncDDGS instead.
  with DDGS() as ddgs:


[snippet: It's important for us to eat like locals so we've made it our mission to find the best Seoul restaurants serving the most delicious and authentic examples of popular Korean dishes like BBQ, bibimbap, kalguksu, naengmyeon, and many others., title: 25 Seoul Restaurants You'll Want to Fly For | Will Fly for Food, link: https://www.willflyforfood.net/seoul-food-guide-25-must-eat-restaurants-in-seoul-south-korea/], [snippet: 1 Seoul is a city renowned for its vibrant and diverse culture, and Korean cuisine is at the heart of it all. From traditional street food to innovative modern eateries, Seoul offers a great variety of dining options for visitors and locals alike., title: 10 Best Korean Restaurants in Seoul - Japan Web Magazine, link: https://jw-webmagazine.com/best-korean-restaurants-in-seoul/], [snippet: Mapo Jeong Daepo 마포정대포 My go-to place is still Jeong Daepo in the Mapo neighborhood, near Gongdeok Station. The whole area is full of BBQ aromas. Most all the restaurants in

In [7]:
!pip install -U duckduckgo-search

In [9]:
print(web_fetch_tool.name)
print(web_fetch_tool.description)
print(web_fetch_tool.args)

web_fetch_tool
web_fetch_tool(url: str) -> str - Useful to fetches the contents of a web page
{'url': {'title': 'Url', 'type': 'string'}}


In [7]:
import langchain

print(langchain.__version__)

0.1.0
